<a href="https://colab.research.google.com/github/jakartaresearch/quora-question-pairs/blob/master/notebook/R4_word2vec_cosine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive/"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive

import pickle
import os
import glob
import pandas as pd
import nltk
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, accuracy_score
from tqdm import tqdm_notebook, tqdm
tqdm_notebook().pandas()

import sys
sys.path.append('drive/My Drive/Colab Notebooks/quora-question-pairs/notebook/')
import cleansing

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
data_path = 'drive/My Drive/Colab Notebooks/quora-question-pairs/data/'
kfold_folder = glob.glob(data_path+'cross_validation_data/*')
train_id_file = '/train_id.csv'
val_id_file = '/val_id.csv'
w2v_embed_path = data_path+'w2v_embed.pkl'

In [0]:
def read_csv(path):
    data = pd.read_csv(path)
    return data

def remove_row_nan(df):
    df = df.dropna(axis = 0)
    return df

In [0]:
def question_similarity(row):
    from sklearn.metrics.pairwise import cosine_similarity 
    
    q1_weight = []
    q2_weight = []

    q1 = row['clean_question1']
    q2 = row['clean_question2']
    
    for word in str(q1).split():
        q1_weight.append(weights.get(word, np.zeros(300)))
    for word in str(q2).split():
        q2_weight.append(weights.get(word, np.zeros(300)))

    q1_weight = np.sum(q1_weight, axis=0)
    q1_weight = np.divide(q1_weight, len(q1.split()))
    q1_weight = q1_weight.reshape(1,-1)

    q2_weight = np.sum(q2_weight, axis=0)
    q2_weight = np.divide(q2_weight, len(q2.split()))
    q2_weight = q2_weight.reshape(1,-1)
    
    sim = cosine_similarity(q1_weight, q2_weight)
    return sim

In [0]:
data = read_csv(data_path+'clean_quora_duplicate_questions.csv')
data = remove_row_nan(data)

In [0]:
weights = pickle.load(open(w2v_embed_path,'rb'))

In [0]:
total_acc = []
total_f1 = []
total_prec = []

for kf, path in enumerate(kfold_folder,1):
    print('Load KFold data from =', path)
    print('KFold -',kf)
    train_id = read_csv(path + train_id_file)
    val_id = read_csv(path + val_id_file)

    # Get specific data by id
    train = data[data.id.isin(train_id.id.values)]
    val = data[data.id.isin(val_id.id.values)]

    # random sample data
    train = train.sample(frac=1, random_state=42)
    val = val.sample(frac=1, random_state=42)

    # calculate cosine similarity
    print('calculate cosine similarity')
    x_train = train.progress_apply(question_similarity, axis=1)
    x_val = val.progress_apply(question_similarity, axis=1)

    # Prepare train, val data
    x_train = x_train.values.reshape(-1, 1)
    x_val = x_val.values.reshape(-1, 1)
    y_train = train.is_duplicate.values
    y_val = val.is_duplicate.values

    # Classifier
    clf = LogisticRegression()
    clf.fit(x_train, y_train)
    # Prediction
    y_pred = clf.predict(x_val)
    # Calculate accuracy
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    prec = precision_score(y_val, y_pred)

    print('KFold -',kf, 'Accuracy :', accuracy)
    print('KFold -',kf, 'F1 :', f1)
    print('KFold -',kf, 'Precision :', prec)
    print('')
    total_acc.append(accuracy)
    total_f1.append(f1)
    total_prec.append(prec)

print('')
print('Accuracy :', sum(total_acc)/5)
print('F1 :', sum(total_f1)/5)
print('Precision :', sum(total_prec)/5)

Load KFold data from = drive/My Drive/Colab Notebooks/quora-question-pairs/data/cross_validation_data/5
KFold - 1
calculate cosine similarity



KFold - 1 Accuracy : 0.6714040100436627
KFold - 1 F1 : 0.5171047369760425
KFold - 1 Precision : 0.5652968762419521

Load KFold data from = drive/My Drive/Colab Notebooks/quora-question-pairs/data/cross_validation_data/3
KFold - 2
calculate cosine similarity



KFold - 2 Accuracy : 0.6686169489009561
KFold - 2 F1 : 0.5126432599599782
KFold - 2 Precision : 0.5609746386909265

Load KFold data from = drive/My Drive/Colab Notebooks/quora-question-pairs/data/cross_validation_data/2
KFold - 3
calculate cosine similarity



KFold - 3 Accuracy : 0.671540431216833
KFold - 3 F1 : 0.5180506398039749
KFold - 3 Precision : 0.5653672450677443

Load KFold data from = drive/My Drive/Colab Notebooks/quora-question-pairs/data/cross_validation_data/1
KFold - 4
calculate cosine similarity



KFold - 4 Accuracy : 0.6710161777074161
KFold - 4 F1 : 0.5154038149720345
KFold - 4 Precision : 0.5649864195558396

Load KFold data from = drive/My Drive/Colab Notebooks/quora-question-pairs/data/cross_validation_data/4
KFold - 5
calculate cosine similarity



KFold - 5 Accuracy : 0.6703236898415564
KFold - 5 F1 : 0.5144990892531877
KFold - 5 Precision : 0.5638374321303098


Accuracy : 0.6705802515420849
F1 : 0.5155403081930435
Precision : 0.5640925223373545
